# bigram LSTM

Model for predicting text sequences a bigram at a time

In [2]:
# Modules
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

### Downloading and Formatting Data

In [7]:
def get_text8(expected_bytes):
    filename = "text8.zip"
    url = "http://mattmahoney.net/dc/"
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    if os.stat(filename).st_size == expected_bytes:
        print('Found and verified')
    else:
        raise Exception('Failed to verify. Try accessing through browser.')
    return filename

def read_text(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()
    return
   
expected_bytes = 31344016
filename = get_text8(expected_bytes)
text = read_text(filename)

print('Total characters: %d' % len(text))
print('Sample: %s' % text[:100])

Found and verified
Total characters: 100000000
Sample:  anarchism originated as a term of abuse first used against early working class radicals including t
